In [ ]:
pip install -U selenium

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import re
import pandas as pd
import numpy as np

In [2]:
USER_LOGIN = '**********'
USER_PASSWORD = '**********'

In [3]:
caps = DesiredCapabilities().CHROME
driver = webdriver.Chrome()
driver.get("https://linkedin.com/uas/login")

# waiting for the page to load
time.sleep(3.5)

# entering username
username = driver.find_element(By.ID, "username")

# In case of an error, try changing the element
# tag used here.

# Enter Your Email Address
username.send_keys(USER_LOGIN)

# entering password
pword = driver.find_element(By.ID, "password")
# In case of an error, try changing the element
# tag used here.

# Enter Your Password
pword.send_keys(USER_PASSWORD)

# Clicking on the log in button
# Format (syntax) of writing XPath -->
# //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

In [4]:
def get_users (link):
    # Open search page
    driver.get(link)
    profile_urls = []

    # NUM_PAGES_TO_PARSE = 12
    NUM_PAGES_TO_PARSE = 12

    # Iterate over pages of search results
    # to collect profile urls
    for i in range(NUM_PAGES_TO_PARSE):
        search_result_links = driver.find_elements(By.CLASS_NAME, "app-aware-link")
        for link in search_result_links:
            href = link.get_attribute("href")
            if 'linkedin.com/in' in href:
                profile_urls.append(href)

        # HACK TO SEE NEXT BUTTON BY SELENIUM
        # We scroll down the page to make element visible for Selenium
        SCROLL_PAUSE_TIME = np.random.randint(1, 3)

        last_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("document.body.style.zoom='20%'")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, (document.body.scrollHeight/2));")
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
            driver.execute_script("window.scrollTo(0, (document.body.scrollHeight));")
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        driver.execute_script("document.body.style.zoom='100%'")

        next_button = driver.find_element(By.CLASS_NAME, 'artdeco-pagination__button--next')
        next_button.click()
        time.sleep(5)

    profile_list = list(set(profile_urls))
    return profile_list
    

In [5]:
try:
    users_df = pd.read_csv('users.csv',index_col=[0])
except FileNotFoundError:
    profile_urls = []
    links = ['https://www.linkedin.com/search/results/people/?keywords=Senior%20developer%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%22602%22%2C%22764%22%2C%221383%22%2C%223166%22%2C%2255798%22%2C%223498%22%2C%225171%22%2C%2226904%22%2C%2256683%22%5D&sid=H47',
    'https://www.linkedin.com/search/results/people/?keywords=senior%20analyst%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%22602%22%2C%22764%22%2C%221383%22%2C%223166%22%2C%2255798%22%2C%223498%22%2C%225171%22%2C%2226904%22%2C%2256683%22%5D&sid=p',
    'https://www.linkedin.com/search/results/people/?keywords=lead%20analyst%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%226177%22%2C%22602%22%2C%22764%22%2C%221383%22%2C%223166%22%2C%223498%22%2C%2226904%22%2C%225171%22%2C%2255798%22%2C%2256683%22%5D&sid=tA6',
    'https://www.linkedin.com/search/results/people/?keywords=senior%20engineer%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%226177%22%2C%22602%22%2C%22764%22%2C%221383%22%2C%223166%22%2C%223498%22%2C%2226904%22%2C%225171%22%2C%2255798%22%2C%2256683%22%5D&sid=HRM',
    'https://www.linkedin.com/search/results/people/?keywords=lead%20engineer%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%226177%22%2C%22602%22%2C%22764%22%2C%221383%22%2C%223166%22%2C%223498%22%2C%2226904%22%2C%225171%22%2C%2255798%22%2C%2256683%22%5D&sid=_6z',
    'https://www.linkedin.com/search/results/people/?keywords=senior%20qa%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%2250321%22%2C%22602%22%2C%22764%22%2C%221383%22%2C%226177%22%2C%223166%22%2C%2255798%22%2C%223498%22%2C%225171%22%2C%2226904%22%2C%2256683%22%5D&sid=apf',
    'https://www.linkedin.com/search/results/people/?keywords=senior%20ui%20ux%20designer%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%22207%22%2C%22272%22%2C%22602%22%2C%22764%22%2C%225171%22%2C%2255798%22%2C%226177%22%2C%221383%22%2C%2226904%22%2C%223166%22%2C%223498%22%2C%2250321%22%2C%2256683%22%5D&sid=ySg',
    'https://www.linkedin.com/search/results/people/?keywords=senior%20product%20manager%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%22207%22%2C%22272%22%2C%22602%22%2C%22764%22%2C%225171%22%2C%2255798%22%2C%226177%22%2C%221383%22%2C%2226904%22%2C%223166%22%2C%223498%22%2C%2250321%22%2C%2256683%22%5D&sid=b60',
    'https://www.linkedin.com/search/results/people/?keywords=senior%20project%20manager%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%22207%22%2C%22272%22%2C%22602%22%2C%22764%22%2C%225171%22%2C%2255798%22%2C%226177%22%2C%221383%22%2C%2226904%22%2C%223166%22%2C%223498%22%2C%2250321%22%2C%2256683%22%5D&sid=Xxx',
    'https://www.linkedin.com/search/results/people/?keywords=senior%20data%20scientist%20NOT%20Ukraine%20NOT%20%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%B0&origin=GLOBAL_SEARCH_HEADER&profileLanguage=%5B%22ru%22%5D&serviceCategory=%5B%226177%22%2C%22602%22%2C%22764%22%2C%221383%22%2C%223166%22%2C%223498%22%2C%2226904%22%2C%225171%22%2C%2255798%22%2C%2256683%22%5D&sid=iVI']

    for i in links:
        profile_urls.append(get_users(i))

    users_df = pd.DataFrame({'link': profile_urls})
    # users_df.to_csv('users.csv')
    temp_df = []
    for i in range(len(users_df)):
        for j in users_df['link'][i].split(','):
            temp_df.append(j)

    users_df = pd.DataFrame({'link': temp_df})
    users_df.to_csv('users.csv')
    users_df


In [6]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2096 entries, 0 to 2095
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    2096 non-null   object
dtypes: object(1)
memory usage: 32.8+ KB


In [15]:
def get_and_print_user_experience(driver, posts_url):
    driver.get(posts_url)

    #Simulate scrolling to capture all posts
    SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # We can adjust this number to get more posts
    NUM_SCROLLS = 5

    for i in range(NUM_SCROLLS):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Parsing posts
    src = driver.page_source

    # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')
    # soup.prettify()

    # experience = soup.find_all('div', {'class':'display-flex full-width'})
    # print(experience)

    # find the experience in years 
    xp_in_years = soup.find_all('span', {'class':'t-14 t-normal t-black--light'})
    xp_in_years=str(xp_in_years).replace('<span class="t-14 t-normal t-black--light">',' ')
    xp_in_years=str(xp_in_years).replace('<span aria-hidden="true">','.') 
    xp_in_years=str(xp_in_years).replace('<!-- -->','.')
    xp_in_years=str(xp_in_years).replace('</span>','')
    xp_in_years=str(xp_in_years).replace('<span class="visually-hidden">',' ')
    xp_in_years = xp_in_years.split('.')

    correct_xp=[]
    years_list = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']
    correct_xp=[]
    for i in xp_in_years:
        positive = 0
        if len(i) > 5:
            for j in  years_list:
                if j in i:
                    positive=1
            if positive == 1:
                correct_xp.append(i)
    
    # duplicates drop
    xp_in_years =[]
    for i in range(len(correct_xp)):
        if (i+1)%2 == 0:
            continue
        else: xp_in_years.append(correct_xp[i])

    # split on date and duration
    correct_xp=[]
    for i in xp_in_years:
        correct_xp.append(i.split('·'))
    
    # drop duration
    xp_in_years = []
    for i in correct_xp:
        xp_in_years.append(i[0])

    # find the history of experience in companies
    xp_company = soup.find_all('span', {'class':'t-14 t-normal'})
    xp_company=str(xp_company).replace('<span class="t-14 t-normal">\n','')
    xp_company=str(xp_company).replace('<span aria-hidden="true">','.') 
    xp_company=str(xp_company).replace('<!-- -->','.')
    xp_company=str(xp_company).replace('</span>','')
    xp_company=str(xp_company).replace('<span class="visually-hidden">','')
    xp_company = xp_company.split('.')

    # filtering
    correct_company=[]
    for i in xp_company:
        if len(i) > 5:
            correct_company.append(i)
    
    # duplicates drop
    xp_company =[]
    for i in range(len(correct_company)):
        if (i+1)%2 == 0:
            continue
        else: 
            xp_company.append(correct_company[i])

    # find the history of experience in companies
    xp_position = soup.find_all('div', {'class':'display-flex align-items-center mr1 t-bold'})
    xp_position=str(xp_position).replace('<div class="display-flex align-items-center mr1 t-bold">','')
    xp_position=str(xp_position).replace('<span aria-hidden="true">','.') 
    xp_position=str(xp_position).replace('<!-- -->','.')
    xp_position=str(xp_position).replace('</span>','')
    xp_position=str(xp_position).replace('</div>','')
    xp_position=str(xp_position).replace('<span class="visually-hidden">','')
    xp_position = xp_position.split('.')

    # filtering len >5
    correct_position=[]
    for i in xp_position:
        if len(i) > 5:
            correct_position.append(i)

    # duplicates drop
    xp_position =[]
    for i in range(len(correct_position)):
        if (i+1)%2 == 0:
            continue
        else: 
            xp_position.append(correct_position[i])

    return(xp_in_years,xp_company,xp_position)

In [8]:
def len_append(required_len,df_to_update):
    if required_len > len(df_to_update):
        for i in range(required_len - len(df_to_update)):
            df_to_update.append('')

In [20]:
def get_and_print_profile_xp (driver, profile_url):
    
    driver.get(profile_url)    # this will open the link    

    # Extracting data from page with BeautifulSoup
    src = driver.page_source

    # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')

    # Extracting the HTML of the complete introduction box
    # that contains the name, company name, and the location
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})

    # In case of an error, try changing the tags used here.
    name_loc = intro.find("h1")

    # Extracting the Name
    name = name_loc.get_text().strip()
    # strip() is used to remove any extra blank spaces

    works_at_loc = intro.find("div", {'class': 'text-body-medium'})

    # this gives us the HTML of the tag in which the Company Name is present
    # Extracting the Company Name
    works_at = works_at_loc.get_text().strip()

    # print("Name -->",  name,'\n', "Works At -->", works_at)

    EXPERIENCE_URL_SFX = '/details/experience/'

    # Get current url from browser
    cur_profile_url = driver.current_url

    # Parse experience 
    xp_years_list, xp_company,xp_position = get_and_print_user_experience(driver, cur_profile_url + EXPERIENCE_URL_SFX)

    ttl_len = max(len(xp_years_list),len(xp_company),len(xp_position))

    len_append(ttl_len,xp_years_list)
    len_append(ttl_len,xp_company)
    len_append(ttl_len,xp_position)

    name_list=[]
    link_list = []
    for i in range(ttl_len):
        name_list.append(name)
        link_list.append(cur_profile_url)

    xp_user_df = pd.DataFrame({'name':name_list,
                            'position': xp_position,
                            'experience': xp_years_list, 
                            'company': xp_company,'link':link_list})

    time.sleep(5)

    return(xp_user_df)

In [21]:
# collecting info on users experience
count = 0
for profile_url in users_df.iloc[0:100,0]:
    user_xp = get_and_print_profile_xp(driver, profile_url)
    if count == 0:
        user_xp_df_out = user_xp
    else:
        user_xp_df_out = pd.concat([user_xp_df_out,user_xp])
    count+=1

user_xp_df_out.to_excel('users_and_experience.xlsx')
user_xp_df_out

,name,position,experience,company,link
0,Roman Grinovski,Frontend Engineer,Mar 2023 - Present,Spacelift · Full-time,https://www.linkedin.com/in/roman-grinovski-0a...
1,Roman Grinovski,Frontend Engineer,Oct 2021 - Mar 2023,Via · Self-employed,https://www.linkedin.com/in/roman-grinovski-0a...
2,Roman Grinovski,Software Engineer,Jul 2020 - Oct 2021,UI Bakery,https://www.linkedin.com/in/roman-grinovski-0a...
3,Roman Grinovski,Frontend Engineer,Sep 2019 - Oct 2021,Akveo · Full-time,https://www.linkedin.com/in/roman-grinovski-0a...
4,Roman Grinovski,Software Engineer,Dec 2017 - Aug 2019,Aligned Code,https://www.linkedin.com/in/roman-grinovski-0a...
...,...,...,...,...,...
5,Vardan Galstyan,Web Application Developer,Dec 2013 - Jan 2020,BlueNet Armenia · Full-time,https://www.linkedin.com/in/vardan-galstyan-12...
6,Vardan Galstyan,Senior Frontend Developer,Jan 2018 - May 2018,Self Employed - Freelance · Freelance,https://www.linkedin.com/in/vardan-galstyan-12...
7,Vardan Galstyan,Web Software Developer,Jul 2015 - Jan 2018,Digitain · Full-time,https://www.linkedin.com/in/vardan-galstyan-12...
8,Vardan Galstyan,Junior Web Developer,Sep 2013 - Dec 2013,AIST Global,https://www.linkedin.com/in/vardan-galstyan-12...


In [ ]:
def get_and_print_user_posts(driver, posts_url):
    driver.get(posts_url)

    #Simulate scrolling to capture all posts
    SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # We can adjust this number to get more posts
    NUM_SCROLLS = 5

    for i in range(NUM_SCROLLS):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Parsing posts
    src = driver.page_source

    # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')
    # soup.prettify()

    posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')
    # print(posts)

    posts_out = []
    reactions_out = []

    # print(f'Number of posts: {len(posts)}')
    for post_src in posts:
        post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})
        

        # if post_text_div is None:
        #     print(post_src)

        if post_text_div is not None:
            post_text = post_text_div.find('span', {'dir': 'ltr'})
        else:
            post_text = None
            posts_out.append(post_text)

        # If post text is found
        if post_text is not None:
            post_text = post_text.get_text().strip()
            posts_out.append(post_text)
            # print(f'Post text: {post_text}')

        reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__reactions-count'})

        # If number of reactions is written as text
        # It has different class name
        if reaction_cnt is None:
            reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__social-proof-text'})
            reactions_out.append(0)

        if reaction_cnt is not None:
            reaction_cnt = reaction_cnt.get_text().strip()
            reactions_out.append(reaction_cnt)
            # print(f'Reactions: {reaction_cnt}')

    return posts_out,reactions_out

In [ ]:
def get_and_print_profile_info(driver, profile_url):
    driver.get(profile_url)        # this will open the link

    # Extracting data from page with BeautifulSoup
    src = driver.page_source

    # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')

    # Extracting the HTML of the complete introduction box
    # that contains the name, company name, and the location
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})

    # In case of an error, try changing the tags used here.
    name_loc = intro.find("h1")

    # Extracting the Name
    name = name_loc.get_text().strip()
    # strip() is used to remove any extra blank spaces

    works_at_loc = intro.find("div", {'class': 'text-body-medium'})

    # this gives us the HTML of the tag in which the Company Name is present
    # Extracting the Company Name
    works_at = works_at_loc.get_text().strip()

    # print("Name -->",  name,'\n', "Works At -->", works_at)

    POSTS_URL_SUFFIX = 'recent-activity/all/'

    time.sleep(5)

    # Get current url from browser
    cur_profile_url = driver.current_url

    # print(cur_profile_url+POSTS_URL_SUFFIX)

    # Parse posts
    posts, reactions = get_and_print_user_posts(driver, cur_profile_url + POSTS_URL_SUFFIX)
    # print(posts, reactions)

    if len(posts) > len(reactions):
        ttl_len = len(posts)
    else:
        ttl_len = len(reactions)

    name_list=[]
    works_at_list = []
    link_list = []
    for i in range(ttl_len):
        name_list.append(name)
        works_at_list.append(works_at)
        link_list.append(cur_profile_url + POSTS_URL_SUFFIX)

    posts_df = pd.DataFrame({'name':name_list,'works_at':works_at_list, 'posts': posts, 'reactions': reactions})
    return(posts_df)

In [ ]:
# Parse profile urls
count = 0
for profile_url in users_df.iloc[0:10,0]:
    user_posts = get_and_print_profile_info(driver, profile_url)
    if count == 0:
        posts_df_out = user_posts
    else:
        posts_df_out = pd.concat([posts_df_out,user_posts])
    time.sleep(2)
    count+=1

posts_df_out
posts_df_out.to_excel('posts.xlsx')

# close the Chrome browser
driver.quit()


In [ ]:
posts_df_out.reset_index()